In [2]:
import pandas as pd
import yfinance as yf

from sklearn.model_selection import train_test_split
from skfolio import Population, RiskMeasure
from skfolio.optimization import InverseVolatility, RiskBudgeting
from skfolio.preprocessing import prices_to_returns

In [3]:
prices = yf.download(['AAPL','MSFT','GOOG','AMZN','META','NVDA','TSLA'], start = '2013-01-01', end = '2024-07-25', progress= False)
prices = prices['Adj Close']

In [4]:
x = prices_to_returns(prices)
x_train,x_test = train_test_split(x, test_size=0.75, shuffle = False)
x_train.head()

Ticker,AAPL,AMZN,GOOG,META,MSFT,NVDA,TSLA
Date,,,,,,,
2013-01-03,-0.012622,0.004547,0.000581,-0.008214,-0.013396,0.000786,-0.016685
2013-01-04,-0.027854,0.002592,0.019760,0.035650,-0.018716,0.032993,-0.010642
2013-01-07,-0.005883,0.035925,-0.004363,0.022949,-0.001870,-0.028897,-0.001744
2013-01-08,0.002692,-0.007748,-0.001973,-0.012237,-0.005246,-0.021927,-0.019220
2013-01-09,-0.015629,-0.000113,0.006573,0.052650,0.005650,-0.022418,-0.001187


In [5]:
model = RiskBudgeting(risk_measure= RiskMeasure.VARIANCE,
                      portfolio_params=dict(name='Risk Parity - Variance'))
model.fit(x_train)
model.weights_

array([0.18227969, 0.12927044, 0.15557646, 0.11372137, 0.17141387,
       0.15757417, 0.09016402])

In [6]:
benchmark = InverseVolatility(portfolio_params=dict(name = 'Inverse Volatility'))
benchmark.fit(x_train)
benchmark.weights_

array([0.1655935 , 0.13636408, 0.17381777, 0.11478181, 0.17543483,
       0.15440734, 0.07960067])

In [7]:
pred_model = model.predict(x_test)
pred_bench = benchmark.predict(x_test)
print(pred_model.annualized_sharpe_ratio)
print(pred_bench.annualized_sharpe_ratio)

1.24306180529019
1.2327799378824775


In [24]:
population = Population([pred_model, pred_bench])
population.plot_cumulative_returns()


In [25]:
population.plot_composition()

In [26]:
population.summary()

,Risk Parity - Variance,Inverse Volatility
Mean,0.14%,0.14%
Annualized Mean,34.97%,34.49%
Variance,0.031%,0.031%
Annualized Variance,7.91%,7.83%
Semi-Variance,0.017%,0.017%
Annualized Semi-Variance,4.25%,4.20%
Standard Deviation,1.77%,1.76%
Annualized Standard Deviation,28.13%,27.98%
Semi-Deviation,1.30%,1.29%
Annualized Semi-Deviation,20.61%,20.49%
